In [6]:
#import all libraries
from sklearn.ensemble import RandomForestClassifier
from all_functions import *
from joblib import dump, load
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV,cross_val_score

In [2]:
feature_selection='ANOVA'
Training='2017'
Features=9
Index_2016,Color_index_2016,All_columns,Maturity_2016,Final_Data_2016,Final_Data_Selected_2016=datacreate_2016(file='C:/Users/tusha/Downloads/Peanut_Maturity.csv'\
                                                                  ,path1='C:/All/Peanut_Maturity_Classification',W_select=199)
Index_2017,Color_index_2017,Maturity_2017,Final_Data_2017,Final_Data_Selected_2017=datacreate_2017(file='C:/Users/tusha/Downloads/Peanut_Maturity_.csv'\
                                                                  ,path2='C:/All/Peanut_Maturity_Classification_2017',W_select=199)

if Training=='2016':
    X_train_data,y_train_data,Train_Color_Index,Train_Index=Final_Data_Selected_2016,Maturity_2016,Color_index_2016,Index_2016
    X_test_data,y_test_data,Test_Color_Index,Test_Index=Final_Data_Selected_2017,Maturity_2017,Color_index_2017,Index_2017
elif Training=='2017':
    X_train_data,y_train_data,Train_Color_Index,Train_Index=Final_Data_Selected_2017,Maturity_2017,Color_index_2017,Index_2017
    X_test_data,y_test_data,Test_Color_Index,Test_Index=Final_Data_Selected_2016,Maturity_2016,Color_index_2016,Index_2016


if feature_selection=='Strong_Feature':
    X_train,X_test,y_train,y_test=Strong_feature_train_test_dataset_create(Features=Features,X_train=X_train_data,\
                                                        y_train=y_train_data,X_test=X_test_data,\
                                                        y_test=y_test_data)
elif feature_selection=='ANOVA':
    X_train,X_test,y_train,y_test=anova_train_test_dataset_create(Features=Features,X_train=X_train_data,\
                                                        y_train=y_train_data,X_test=X_test_data,\
                                                        y_test=y_test_data)    
elif feature_selection=='Max':
    X_train,X_test,y_train,y_test=Max_feature_train_test_dataset_create(Features=Features,X_train=X_train_data,\
                                                        y_train=y_train_data,X_test=X_test_data,\
                                                        y_test=y_test_data)  

elif feature_selection=='None':
    X_train,X_test,y_train,y_test=X_train_data,X_test_data,y_train_data,y_test_data      

In [7]:
#Set the model Random Forest classifier
"""
Make a pipline of preprocessing, a grid of parameters for the classifier, fit the model, evaluate the model (Writing into a text file) 
"""
file = open("Result_RF_v2.txt", "w")
pipe_rf = make_pipeline(StandardScaler(),PCA(n_components=10),RandomForestClassifier());
max_depth = np.arange(8,20,2)
np.append(max_depth,None)
parameter_grid_rf = [{'randomforestclassifier__n_estimators': np.arange(10,20,2),
               'randomforestclassifier__max_features': ['auto', 'sqrt'],
               'randomforestclassifier__max_depth': max_depth,
               'randomforestclassifier__min_samples_split': [2, 5, 10],
               'randomforestclassifier__min_samples_leaf': [4,5,6,7,9],
               'randomforestclassifier__bootstrap': [True, False],
               'randomforestclassifier__random_state': [1]}]
grf = GridSearchCV(estimator=pipe_rf,param_grid=parameter_grid_rf, scoring='accuracy',cv=10,refit=True,n_jobs=-1)
grf = grf.fit(X_train, y_train)

In [8]:
Score_train=grf.score(X_train, y_train)
Score_test=grf.score(X_test, y_test)